In [ ]:
from functions_plots import *
from functions_data_transformation import load_suite2p_paths as load_s2p

def getStats(stat, frame_shape, output_df):
    """Accesses suite2p stats on ROIs and filters ROIs based on cascade spike probability being >= 1 into nid2idx and nid2idx_rejected (respectively)"""
    MIN_PROB = 0 
    pixel2neuron = np.full(frame_shape, fill_value=np.nan, dtype=float)
    scatters = dict(x=[], y=[], color=[], text=[], iscell = [])
    nid2idx = {}
    nid2idx_rejected = {}
    print(f"Number of detected ROIs: {stat.shape[0]}")
    for n in range(stat.shape[0]):
        estimated_spikes = output_df.iloc[n]["EstimatedSpikes"]

        if estimated_spikes > MIN_PROB:
            nid2idx[n] = len(scatters["x"]) # Assign new idx
            
        else:
            nid2idx_rejected[n] = len(scatters["x"])

        ypix = stat.iloc[n]['ypix'].flatten() - 1 #[~stat.iloc[n]['overlap']] - 1
        xpix = stat.iloc[n]['xpix'].flatten() - 1 #[~stat.iloc[n]['overlap']] - 1

        valid_idx = (xpix>=0) & (xpix < frame_shape[1]) & (ypix >=0) & (ypix < frame_shape[0])
        ypix = ypix[valid_idx]
        xpix = xpix[valid_idx]
        yext, xext = boundary(ypix, xpix)
        scatters['x'] += [xext]
        scatters['y'] += [yext]
        pixel2neuron[ypix, xpix] = n

    return scatt ers, nid2idx, nid2idx_rejected, pixel2neuron


def dispPlot(MaxImg, scatters, nid2idx, nid2idx_rejected,
             pixel2neuron, F, Fneu, save_path, axs=None):
             if axs is None:
                fig = plt.figure(constrained_layout=True)
                NUM_GRIDS=12
                gs = fig.add_gridspec(NUM_GRIDS, 1)
                ax1 = fig.add_subplot(gs[:NUM_GRIDS-2])
                fig.set_size_inches(12,14)
             else:
                 ax1 = axs
                 ax1.set_xlim(0, MaxImg.shape[0])
                 ax1.set_ylim(MaxImg.shape[1], 0)
             ax1.imshow(MaxImg, cmap='gist_gray')
             ax1.tick_params(axis='both', which='both', bottom=False, top=False, 
                             labelbottom=False, left=False, right=False, labelleft=False)
             print("Neurons count:", len(nid2idx))
             norm = matplotlib.colors.Normalize(vmin=0, vmax=1, clip=True) 
             mapper = cm.ScalarMappable(norm=norm, cmap=cm.gist_rainbow) 

             def plotDict(n2d2idx_dict, override_color = None):
                 for neuron_id, idx in n2d2idx_dict.items():
                     color = override_color if override_color else mapper.to_rgba(scatters['color'][idx])
                            # print(f"{idx}: {scatters['x']} - {scatters['y'][idx]}")
                            
                     sc = ax1.scatter(scatters["x"][idx], scatters['y'][idx], color = color, 
                                      marker='.', s=1)
             plotDict(nid2idx, 'g')
             plotDict(nid2idx_rejected, 'm')
             ax1.set_title(f"{len(nid2idx)} neurons used (green) out of {len(nid2idx)+len(nid2idx_rejected)} neurons detected (magenta - rejected)") 

             plt.savefig(save_path)
             plt.close(fig)

In [47]:
home = r'D:\users\JC\pipeline\cysteine toxicity\002-HCO3\240704_L-cys_HCO3\plate01\t10_naoh\240704_wtRt_C2006_DIV14_100k_bc_plate01_10x_treat_well001\suite2p\plane0'
ops = np.load(home + '\\' + 'ops.npy', allow_pickle = True).item()
stats = np.load(home + '\\' + 'stat.npy', allow_pickle = True)
iscell = np.load(home + '\\' + 'iscell.npy', allow_pickle = True)
cascade = np.load(home + '\\' + 'predictions_deltaF.npy', allow_pickle = True)
flat_cell = iscell.flatten()[0:-1:2]
Img = getImg(ops)
test = cascade[:, ~np.isnan(cascade).any(axis=0)]
estimated_spikes = []
for t in test:
    estimated_spikes.append(np.sum(t))

idx_true = []
for cell in estimated_spikes:
    if cell > 0:
        idx_true.append(estimated_spikes.iloc(cell))

idx_true
# scatters,nid2idx, idx_reject, pixel2n = getStats(stats, Img.shape, output_df)


AttributeError: 'list' object has no attribute 'iloc'